## 3. Model m1
- With sklearn, several pipelines are built which combine further preprocessing, feature selection, dimensionality reduction, and a modeling algorithm.
- Computations are run with dask, through the joblib backend.

### Setup

In [1]:
from datetime import datetime

from dask.distributed import Client
import joblib

import pandas as pd
import numpy as np

from sklearn.neighbors import LocalOutlierFactor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression

from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()

In [ ]:
class Feature_reader():
    """
    Each feature has a first name, optionally a second name, a statistic and a number.
    This class allows to group feautures according to these aspects or combinations of these aspects.
    Each method produces a list of feature names or a list of lists of feature names.
    """

    def __init__(self, csv):
        self.fts = pd.read_csv(csv, dtype={'n':"string"})
        self.fts = self.fts.fillna('')

    def format(self, select):
        return select.apply(lambda x: '_'.join(x).replace('__', '_'), axis=1).tolist()

    def all(self):
        select = self.fts.copy()
        return self.format(select)

    def first(self):
        select = self.fts.copy()
        select = select.loc[select['n']=='01']
        return self.format(select)

    def min(self):
        select = self.fts.copy()
        select = select.loc[select['stat']=='min']
        return self.format(select)
    
    def max(self):
        select = self.fts.copy()
        select = select.loc[select['stat']=='max']
        return self.format(select)

    def median(self):
        select = self.fts.copy()
        select = select.loc[select['stat']=='median']
        return self.format(select)

    def mean(self):
        select = self.fts.copy()
        select = select.loc[select['stat']=='mean']
        return self.format(select)
    
    def std(self):
        select = self.fts.copy()
        select = select.loc[select['stat']=='std']
        return self.format(select)

    def skew(self):
        select = self.fts.copy()
        select = select.loc[select['stat']=='skew']
        return self.format(select)

    def kurtosis(self):
        select = self.fts.copy()
        select = select.loc[select['stat']=='kurtosis']
        return self.format(select)

    def per_nns(self):
        """
        List of lists per name1, name2, stat, per name1, name2.
        """
        select = self.fts.copy()
        select = [[self.format(grp2) for idx2, grp2 in grp.groupby(by=['stat'], sort=False)] for idx, grp in select.groupby(by=['name1', 'name2'])]
        return select
    
    def per_sn(self):
        """
        List of lists per stat, n. 
        """
        select = self.fts.copy()
        select = [[self.format(grp2) for idx2, grp2 in grp.groupby(by=['n'], sort=False)] for idx, grp in select.groupby(by=['stat'], sort=False)]
        return select

fts = Feature_reader('features.csv')

In [ ]:
y = pd.read_csv(f'featsets/y1.csv', header=None)
X = pd.read_csv(f'featsets/X1.csv', delimiter=',', header=None)

In [21]:
client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 8,Total memory: 16.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:52152,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 16.00 GiB
Comm: tcp://127.0.0.1:52163,Total threads: 2
Dashboard: http://127.0.0.1:52167/status,Memory: 4.00 GiB
Nanny: tcp://127.0.0.1:52158,


### Linear regression

In [ ]:
measures = ['r2', 'neg_mean_squared_error', 'neg_root_mean_squared_error', 'neg_mean_absolute_error']

In [22]:
def frmt(df):
    
    df = df[['params', 'mean_test_r2', 'mean_test_neg_mean_squared_error', 'mean_test_neg_root_mean_squared_error','mean_test_neg_mean_absolute_error']]
    df = df.rename(columns={
        'mean_test_r2': 'r2',
        'mean_test_neg_mean_squared_error': 'mse',
        'mean_test_neg_root_mean_squared_error': 'rmse',
        'mean_test_neg_mean_absolute_error': 'mae'
    })

    df['r2'] = df['r2'].round(3)
    df['mse'] = df['mse'].abs().round(3)
    df['rmse'] = df['rmse'].abs().round(3)
    df['mae'] = df['mae'].abs().round(3)

    df = df.set_index('params')

    return df

#### Linear regression with all fts

- Initial results, with unscaled target (not yet a box cox applied), were:

|param|r2|mse|rmse|mae|
|-|-|-|-|-|
|intercept=true|0.017|6.272885e+07|7905.048|2520.407|
|intercept=false|0.017|6.273949e+07|7905.764|2520.480|

Pretty bad.

In [17]:
with joblib.parallel_backend('dask'):
    pipeline = Pipeline([
        ('reg', LinearRegression()),
    ])
    # print(pipeline.get_params().keys())

    parameters = {
        'reg__fit_intercept': [False, True]
    }

    grid_search = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, scoring=measures, refit=False)
    grid_search.fit(X, y)

    results = pd.DataFrame(grid_search.cv_results_)

frmt(results)

,r2,mse,rmse,mae
params,,,,
{'reg__fit_intercept': False},0.085,1.533,1.238,0.962
{'reg__fit_intercept': True},0.083,1.537,1.239,0.962


#### Linear regression with pca reduction

In [18]:
with joblib.parallel_backend('dask'):
    pipeline = Pipeline([
        ('pca', PCA()),
        ('reg', LinearRegression()),
    ])
    # print(pipeline.get_params().keys())

    parameters = {
        'pca__n_components': [5, 10, 20],
    }

    grid_search = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, scoring=measures, refit=False)
    grid_search.fit(X, y)

    results = pd.DataFrame(grid_search.cv_results_)

frmt(results)

,r2,mse,rmse,mae
params,,,,
{'pca__n_components': 5},0.011,1.658,1.288,1.023
{'pca__n_components': 10},0.017,1.648,1.284,1.020
{'pca__n_components': 20},0.022,1.639,1.280,1.018


#### Linear regression with pca reduction and polynomial fts

- Computing polynomial keeps running into compute errors. 
- First try was `parameters = {'pca__n_components': [5, 10, 20], 'poly__degree': [2, 5, 10]}`, which ran into trouble.
- Considering the shape of the data as visualized in the PCA in the exploration phase, for a pca to two dimensions, a polynomial of two degrees would do.

In [19]:
with joblib.parallel_backend('dask'):
    pipeline = Pipeline([
        ('pca', PCA()),
        ('poly', PolynomialFeatures()),
        ('reg', LinearRegression()),
    ])

    parameters = {
        'pca__n_components': [5, 10, 20],
        'poly__degree': [2]
    }

    grid_search = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, scoring=measures, refit=False)
    grid_search.fit(X, y)

    results = pd.DataFrame(grid_search.cv_results_)

frmt(results)

,r2,mse,rmse,mae
params,,,,
"{'pca__n_components': 5, 'poly__degree': 2}",0.018,1.646,1.283,1.019
"{'pca__n_components': 10, 'poly__degree': 2}",0.034,1.619,1.272,1.011
"{'pca__n_components': 20, 'poly__degree': 2}",0.015,1.651,1.285,1.006


### Ridge regression
- https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html#sklearn.linear_model.Ridge


### Lasso

- https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html#sklearn.linear_model.Lasso
- https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LassoLarsCV.html#sklearn.linear_model.LassoLarsCV

### Tree regression

- https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html
- https://scikit-learn.org/stable/modules/generated/sklearn.tree.ExtraTreeRegressor.html#sklearn.tree.ExtraTreeRegressor
- https://george-jen.gitbook.io/data-science-and-apache-spark/decision-tree-regression

### Close client

In [27]:
client.close()